In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

In [2]:
train_png = sorted(glob('data/train/*.png'))
test_png = sorted(glob('data/test/*.png'))

In [3]:
train_y = pd.read_csv("data/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]


In [4]:
# train, test 이미지 리사이즈
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img


In [5]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████████████████████████████████████████████████████████████████████████| 2154/2154 [00:40<00:00, 53.53it/s]


In [6]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
batch_size = 32
epochs = 25

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [8]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

# 최적화
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# 손실함수
criterion = nn.CrossEntropyLoss()
# 특성별로 데이터 스케일이 다를 경우, 모든 특성의 범위를 갖게 만들어줌 
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to C:\Users\user/.cache\torch\hub\checkpoints\mobilenetv3_large_100_ra-f55367f5.pth


epoch : 1/25    time : 30s/723s
TRAIN    loss : 1.00475    f1 : 0.24352
epoch : 2/25    time : 26s/602s
TRAIN    loss : 0.51179    f1 : 0.40404
epoch : 3/25    time : 26s/579s
TRAIN    loss : 0.41850    f1 : 0.50646
epoch : 4/25    time : 26s/552s
TRAIN    loss : 0.28715    f1 : 0.60912
epoch : 5/25    time : 26s/526s
TRAIN    loss : 0.22116    f1 : 0.71030
epoch : 6/25    time : 26s/500s
TRAIN    loss : 0.21802    f1 : 0.72989
epoch : 7/25    time : 26s/473s
TRAIN    loss : 0.20202    f1 : 0.74954
epoch : 8/25    time : 26s/447s
TRAIN    loss : 0.18166    f1 : 0.76653
epoch : 9/25    time : 26s/422s
TRAIN    loss : 0.16777    f1 : 0.78062
epoch : 10/25    time : 26s/395s
TRAIN    loss : 0.13618    f1 : 0.83742
epoch : 11/25    time : 26s/368s
TRAIN    loss : 0.10820    f1 : 0.85109
epoch : 12/25    time : 26s/343s
TRAIN    loss : 0.12346    f1 : 0.84709
epoch : 13/25    time : 27s/320s
TRAIN    loss : 0.11161    f1 : 0.88203
epoch : 14/25    time : 27s/292s
TRAIN    loss : 0.09599    

In [9]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [10]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [11]:
submission = pd.read_csv("data/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-misplaced
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-poke_insulation


In [12]:
submission.to_csv("baseline.csv", index = False)